In [1]:
# libraries required, please install pandas
import pandas as pd
from unidecode import unidecode
import re
import datetime
from datetime import date
import json
import numpy as np
import os
import time
# set column options
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [2]:
j_data=pd.read_pickle('j_data_final.pkl')
cit_network=pd.read_pickle('network_cit_1940_2020.pkl')
all_refs=pd.read_pickle('refs_1940_2020.pkl')
relevant=pd.read_pickle('refs_1940_2020_top5.pkl')

In [3]:
ids=list(set(cit_network['id_o'].astype(str)))
ids_m=list(set(cit_network['match_id'].astype(str)))



In [4]:
all_refs.shape

(534347, 31)

In [5]:
all_refs.columns

Index(['ref_ord', 'id_o', 'year_o', 'type', 'author', 'title', 'journal',
       'year', 'volume', 'issue', 'pages', 'chapter_title', 'location',
       'publisher', 'text_full', 'jstor', 'journal_proc', 'year_proc',
       'year_latest', 'volume_proc', 'issue_proc', 'title_proc', 'match_url',
       'outcome', 'match_id', 'f_key', 'authors', 'month', 'full_reference',
       'status', 'alt_j'],
      dtype='object')

In [6]:
all_refs['pipeline_origin']=all_refs["f_key"].str.split("_").str[0]

In [7]:
all_refs[["pipeline_origin"]].value_counts()

pipeline_origin
tesseract          480867
mturk               53480
Name: count, dtype: int64

In [8]:
nodes=j_data[(j_data['id_o'].isin(ids)==True)|(j_data['id_o'].isin(ids_m)==True)][['id_o','journal', 
       'year', 'content_type']].drop_duplicates(keep='first').reset_index(drop=True)
nodes['id_o']=nodes['id_o'].astype(str)

In [10]:
nodes=nodes.rename(columns={'id_o':"Id"})
nodes.to_csv("nodes_alt_header.csv", index=None, encoding="UTF-8")


In [11]:
nodes.head()

,Id,journal,year,content_type
0,26966478,The American Economic Review,2020,Article
1,26966479,The American Economic Review,2020,Article
2,26966480,The American Economic Review,2020,Article
3,26966481,The American Economic Review,2020,Article
4,26966482,The American Economic Review,2020,Article


In [12]:
cit_network["match_id"]=cit_network["match_id"].astype(str)
cit_network["id_o"]=cit_network["id_o"].astype(str)

In [13]:
edges=cit_network[(cit_network["id_o"]!=cit_network["match_id"]) & (cit_network["match_id"]!="CHECKREQ")].drop_duplicates(subset=['match_id',"id_o"], keep="first")

In [14]:
edges=edges.rename(columns={'id_o':"Source", 'match_id':'Target'})
edges.to_csv('edges_alt_header.csv', index=False)

In [15]:
nodes.shape

(24442, 4)

In [16]:
edges.shape

(107247, 3)

In [17]:
cit_network.shape

(108975, 3)

In [18]:
checks=list(cit_network[cit_network['match_id']=="CHECKREQ"]['id_o'])

In [19]:
inds=list(j_data[(j_data['year']>=1940)&(j_data['content_type']!='MISC')&(j_data['content_type']!='Review')]['id_o'])

In [20]:
n_ind=list(set(ids+ids_m+checks))

In [21]:
len(n_ind)

24443

In [22]:
len(inds)

30519

In [23]:
test=j_data[(j_data['id_o'].isin(n_ind)==False)&(j_data['content_type']!='MISC')&(j_data['content_type']!='Review')&(j_data['content_type']!='Review2')& (j_data['year']>=1940)]

In [24]:
test['content_type'].value_counts()

content_type
Article       5487
Discussion     536
Comment        366
Reply          341
Errata          66
Rejoinder       64
Correction       3
Name: count, dtype: int64

In [25]:
test['journal'].value_counts()

journal
The American Economic Review          4384
Econometrica                           946
Journal of Political Economy           811
The Quarterly Journal of Economics     447
The Review of Economic Studies         275
Name: count, dtype: int64

In [26]:
# test[test['journal']=="american economic review"][['year', 'content_type']].value_counts()
# test[(test['journal']=="american economic review")&(test['year']==2010)]

list(test['id_o'].shape)

[6863]

In [27]:
j_inds=j_data[(j_data['content_type']!='MISC')&(j_data['content_type']!='Review')&(j_data['content_type']!='Review2')& (j_data['year']>=1940)]

In [28]:
cit_network.shape

(108975, 3)

In [29]:
edges.shape

(107247, 3)

In [30]:
cit_network.columns

Index(['id_o', 'match_id', 'f_key'], dtype='object')

In [31]:
relevant.shape

(108988, 31)

In [32]:
all_refs.shape

(534347, 32)

In [33]:
digi=list(set(all_refs['id_o']))

In [34]:
len(digi)

24043

In [35]:
test[test['id_o'].isin(digi)==False].head()

,issue_url,author,title,journal,volume,number,pages,year,ISSN,abstract,...,scopus_page_end_og,scopus_year_og,s_fix,scopus_pages,scopus_indicator,id,authors_lower,page_count,author_count,id_o
0,https://academic.oup.com//qje/issue/135/3,"Enke, Benjamin",what you see is all there is,The Quarterly Journal of Economics,135,3,1363-1398,2020,0033-5533,"News reports and communication are inherently constrained by space, time, and attention. As a result, news sources often condition the decision of whether to share a piece of information on the similarity between the signal and the prior belief of the audience, which generates a sample selection problem. This article experimentally studies how people form beliefs in these contexts, in particular the mechanisms behind errors in statistical reasoning. I document that a substantial fraction of experimental participants follows a simple “what you see is all there is” heuristic, according to which participants exclusively consider information that is right in front of them, and directly use the sample mean to estimate the population mean. A series of treatments aimed at identifying mechanisms suggests that for many participants, unobserved signals do not even come to mind. I provide causal evidence that the frequency of such incorrect mental models is a function of the computational complexity of the decision problem. These results point to the context dependence of what comes to mind and the resulting errors in belief updating.",...,1398,2020.0,0.0,1363-1398,0,qjaa012,"enke, benjamin",35.0,1,qjaa012
242,https://www.jstor.org/stable/10.2307/e26636969,Guido Ascari and Paolo Bonomolo and Hedibert F. Lopes,walk on the wild side: temporarily unstable paths and multiplicative sunspots,The American Economic Review,109,5,1805-1842,2019,"00028282, 19447981","We propose a generalization of the rational expectations framework to allow for temporarily unstable paths. Our approach introduces multiplicative sunspot shocks and it yields drifting parameters and stochastic volatility. Then, we provide an econometric strategy to estimate this generalized model on the data. The methodology allows the data to choose between different possible alternatives: determinacy, indeterminacy, and temporary instability. We apply our methodology to US inflation dynamics in the 1970s through the lens of a simple New Keynesian model. When temporarily unstable paths are allowed, the data unambiguously select them to explain the stagflation period in the 1970s.",...,1842,2019.0,0.0,1805-1842,0,26636977,guido ascari and paolo bonomolo and hedibert f. lopes,38.0,3,26636977
254,https://www.jstor.org/stable/10.2307/e26637201,Andrey Malenko and Anton Tsoy,selling to advised buyers,The American Economic Review,109,4,1323-1348,2019,"00028282, 19447981","In many cases, buyers are not informed about their valuations and rely on experts, who are informed but biased for overbidding. We study auction design when selling to such “advised buyers.” We show that a canonical dynamic auction, the English auction, has a natural equilibrium that outperforms standard static auctions in expected revenues and allocative efficiency. The ability to communicate as the auction proceeds allows for more informative communication and gives advisors the ability to persuade buyers into overbidding. The same outcome is the unique equilibrium of the English auction when bidders can commit to contracts with their advisors.",...,1348,2019.0,0.0,1323-1348,0,26637207,andrey malenko and anton tsoy,26.0,2,26637207
378,https://www.jstor.org/stable/10.2307/e26528515,Seunghoon Na and Stephanie Schmitt-Grohe and Martin Uribe and Vivian Yue,the twin ds: optimal default and devaluation,The American Economic Review,108,7,1773-1819,2018,"00028282, 19447981","A salient characteristic of sovereign defaults is that they are typically accompanied by large devaluations. This paper presents new evidence of this empirical regularity known as the Twin Ds and proposes a model tha